# Libraries

In [ ]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 78 kB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from catboost import CatBoostRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
from scipy import stats
pd.set_option('display.max_rows', 500)

# Importing Datasets

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Data Science Magic/Projects/Housing Prices Competition/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Data Science Magic/Projects/Housing Prices Competition/test.csv')

In [ ]:
train.shape, test.shape

((1460, 81), (1459, 80))

In [ ]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,...,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,...,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,...,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,G

# Dropping Outliers

In [ ]:
train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index, axis=0 ,inplace=True)

In [ ]:
train.shape

(1458, 81)

# Combining Datasets

In [ ]:
temp = pd.concat([train, test]).reset_index(drop=True)

In [ ]:
temp.shape

(2917, 81)

# Capturing target variable, aswell as ID columns (Won't be used in testing)


In [ ]:
train_ID = train['Id']
test_ID = test['Id']
y = train['SalePrice']

# Feature Engineering

In [ ]:
temp['Total_Sqft'] = temp['TotalBsmtSF'] + temp['1stFlrSF'] + temp['2ndFlrSF']

# Features for Website

In [ ]:
temp = temp[['Total_Sqft', 'OverallQual', 'Neighborhood', 'BsmtQual','MSSubClass']]

In [ ]:
temp.head()

,Total_Sqft,OverallQual,Neighborhood,BsmtQual,MSSubClass
0,2566.0,7,CollgCr,Gd,60
1,2524.0,6,Veenker,Gd,20
2,2706.0,7,CollgCr,Gd,60
3,2473.0,7,Crawfor,TA,70
4,3343.0,8,NoRidge,Gd,60


# Filling Nulls

In [ ]:
temp['Total_Sqft'] = temp['Total_Sqft'].fillna(temp['Total_Sqft'].median())

In [ ]:
temp['BsmtQual'] = temp['BsmtQual'].fillna(value='No Basement')

In [ ]:
temp.isnull().sum()

Total_Sqft      0
OverallQual     0
Neighborhood    0
BsmtQual        0
MSSubClass      0
dtype: int64

# This particular feature was miscategorized

In [ ]:
temp['MSSubClass'] = temp['MSSubClass'].astype(str)

# Simplifying MSSubClass

In [ ]:
temp['MSSubClass'] = temp['MSSubClass'].map(lambda x: '1-STORY' if x=='20' or x=='30' or x=='40' else '1-1/2 STORY' if x=='45' or x=='50' \
                            else '2-STORY' if x=='60' or x=='70' else 'PUD' if x=='120' or x=='150' or x=='160' or x=='180' else 'Other')

In [ ]:
temp['MSSubClass'].value_counts()

1-STORY        1224
2-STORY         701
Other           359
PUD             328
1-1/2 STORY     305
Name: MSSubClass, dtype: int64

# Splitting Dataset back into original rows

In [ ]:
train = temp.iloc[:1458]
train.shape

(1458, 5)

In [ ]:
test = temp.iloc[1458:]
test.shape

(1459, 5)

#Train Test Split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(train, y, test_size=0.20, random_state=0)

# Defining Categoricals for Catboost

In [ ]:
categoricals = ['Neighborhood', 'BsmtQual', 'MSSubClass']

# Checking Over/Under fitting

In [ ]:
# def automation_rmse(model, x, y, xtest, ytest):
#     model.fit(x, y)
#     pred = model.predict(xtest)
#     return np.sqrt(mean_squared_error(pred, ytest))

In [ ]:
# automation_rmse(CatBoostRegressor(cat_features=categoricals, verbose=False, task_type="GPU")\
#     , xtrain, ytrain, xtest, ytest)

In [ ]:
# automation_rmse(CatBoostRegressor(cat_features=categoricals, verbose=False, task_type='GPU')\
#     , xtest, ytest, xtrain, ytrain)

# Pickling

In [ ]:
model =CatBoostRegressor(cat_features=categoricals, verbose=False, task_type="GPU")

In [ ]:
model_fit =model.fit(train, y)

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/Data Science Magic/Projects/Web Application/model_fit.pickle', 'wb') as f:
     pickle.dump(model_fit, f)